In [24]:
import os
import sys
from glob import glob
import numpy as np
import scipy
import matplotlib.pyplot as plt
from tabulate import tabulate
sys.path.insert(1, os.path.realpath(os.path.join(sys.path[0], os.pardir, os.pardir)))
from frequency_response import FrequencyResponse
from constants import ROOT_DIR

In [34]:
def inear_score(fr):
    sl = np.logical_and(fr.frequency >= 20, fr.frequency <= 10000)
    x = fr.frequency[sl]
    xm = np.mean(x)
    y = fr.error[sl]
    ym = np.mean(y)

    slope, _, _, _, _ = scipy.stats.linregress(np.log(x), y)
    mean = np.mean(np.abs(fr.error[np.logical_and(fr.frequency >= 40, fr.frequency <= 10000)]))
    std = np.std(y)
    score = 100.0795 - 8.5 * std - 6.796 * slope - 3.475 * mean
    # TODO: score and std differs from oratory1990 PDFs, could be Harman in-ear 2017-1 target
        
    return score, slope, mean, std


def onear_score(fr):
    sl = np.logical_and(fr.frequency >= 50, fr.frequency <= 10000)
    x = fr.frequency[sl]
    xm = np.mean(x)
    y = fr.error[sl]
    ym = np.mean(y)
    
    slope, _, _, _, _ = scipy.stats.linregress(np.log(x), y)
    std = np.std(y)
    mean = np.mean(np.abs(y))
    score = 114.49 - 12.62 * std - 15.52 * np.abs(slope)
        
    return score, slope, mean, std

In [47]:
harman_oe = FrequencyResponse.read_from_csv(os.path.join(ROOT_DIR, 'compensation', 'harman_over-ear_2018.csv'))
harman_ie = FrequencyResponse.read_from_csv(os.path.join(ROOT_DIR, 'compensation', 'harman_in-ear_2019v2.csv'))
#harman_ie = FrequencyResponse.read_from_csv(os.path.join(ROOT_DIR, 'compensation', 'harman_in-ear_2017-1.csv'))

with open('frequencies.csv', 'r', encoding='utf-8') as fh:
    f = fh.read().strip().split('\n')[::-1]
    f = np.array([float(x) for x in f])

harman_oe.interpolate(f)
harman_ie.interpolate(f)

In [ ]:
onear = []
for fp in glob(os.path.join(ROOT_DIR, 'measurements', 'oratory1990', 'data', 'onear', '*', '*.csv')):
    fr = FrequencyResponse.read_from_csv(fp)
    fr.interpolate(f)
    fr.compensate(harman_oe, bass_boost_gain=0.0)
    score, slope, mean, std = onear_score(fr)
    onear.append([fr.name, f'{score:.0f}', f'{slope:.2f}', f'{mean:.2f}', f'{std:.2f}'])
    # TODO: ignore samples
onear = sorted(onear, key=lambda x: float(x[1]), reverse=True)
    
onear_table = tabulate(
    onear, headers=['Model', 'Score', 'Slope', 'Mean', 'STD'], tablefmt='orgtbl'
).replace('+', '|').replace('|-', '|:')
print(onear_table)

In [48]:
inear = []
names = set()
for db in ['oratory1990', 'crinacle']:
    for fp in glob(os.path.join(ROOT_DIR, 'measurements', db, 'data', 'inear', '*', '*.csv')):
        fr = FrequencyResponse.read_from_csv(fp)
        fr.interpolate(f)
        fr.compensate(harman_ie, bass_boost_gain=0.0)
        score, slope, mean, std = inear_score(fr)
        if fr.name not in names:
            inear.append([fr.name, f'{score:.0f}', f'{slope:.2f}', f'{mean:.2f}', f'{std:.2f}'])
            names.add(fr.name)
            # TODO: ignore samples
inear = sorted(inear, key=lambda x: float(x[1]), reverse=True)

inear_table = tabulate(
    inear, headers=['Model', 'Score', 'Slope', 'Mean', 'STD'], tablefmt='orgtbl'
).replace('+', '|').replace('|-', '|:')
print(inear_table)

| Model                                                |   Score |   Slope |   Mean |   STD |
|:-----------------------------------------------------|:--------|:--------|:-------|:------|
| AKG N5005                                            |      87 |   -0.09 |   1.32 |  1.11 |
| HYLA CE-5                                            |      85 |   -0.35 |   1.67 |  1.4  |
| HYLA CE-5 (sample 2)                                 |      85 |   -0.23 |   1.53 |  1.3  |
| AKG N5005 (high)                                     |      83 |   -0.08 |   1.56 |  1.46 |
| AKG N5005 (mid high)                                 |      83 |   -0.28 |   1.53 |  1.62 |
| HYLA CE-5 (sample 1)                                 |      83 |   -0.46 |   1.89 |  1.61 |
| Intime Sora                                          |      83 |   -0.79 |   1.37 |  2.1  |
| Philips SHE3855                                      |      82 |   -0.74 |   2.12 |  1.91 |
| Samsung Galaxy Buds                                  |    